_Adapted from notebook used by [nbclab.github.io](https://nbclab.github.io)._

# Retrieve new publications from PubMed 

This notebook is used to search for and retrieve latest publications by Dr. Khan using BioPython's PubMed search tool. A publication-specific MarkDown file is generated for each unique paper, with many elements automatically set up. As noted in the original notebook, you generally should check that the link to the markdown file exists. Unfortunately, preprints cannot be found via this method (though they can be added manually). This notebook cannot find new preprints. The process is automated and runs monthly using Github actions.

## Steps (via Github or manual)

1. Run this notebook.
2. If any new papers were grabbed, check the following:
    1. The paper has either of the lab PIs as an author. Ensure that it isn't by *another* AR Khan.
    2. The paper is not a duplicate of a preprint or another version of the paper. If so, merge the two versions.
3. Save the changes to the notebook.
4. Push changes to the notebook and affected files to GitHub.
5. Open a pull request to khanlab/khanlab.github.io

In [1]:
# Libraries
import os 

from Bio import Entrez, Medline
from datetime import datetime
from dateutil import parser
import pandas as pd

### Check existing publications

In [2]:
# First count number of articles from previous grab
pub_data = "_data/publications/publications.csv"

# Update count of publications from existing file
old_count = 0 
if os.path.isfile(pub_data):
    df_old = pd.read_csv(pub_data)
    old_count = len(df_old)

### Perform new query

In [3]:
# Only grab papers from after the lab PI came to UWO
search_criteria = ['''"Khan AR"[AUTH] AND ("2015/01/01"[PDAT] : "3000/12/31"[PDAT]) AND
                    ("Western University"[AFFL] OR "University of Western Ontario"[AFFL] OR
                     "Brain and Mind Institute"[AFFL] OR "Robarts Research Institute"[AFFL])''']

# Email required to search
Entrez.email = ''

In [4]:
rows = []

# Publications to skip (possibly due to another user with same initial)
skip_pmids = [32971934, 29641820, 29634829]
skip_pmids = [str(pmid) for pmid in skip_pmids]

for TERM in search_criteria:
    search = Entrez.esearch(db="pubmed", retmax="2", term=TERM)
    result = Entrez.read(search)
    print(f"Total number of publications containing {TERM}: {result['Count']}")
    
    search_all = Entrez.esearch(db="pubmed", term=TERM, retmax=result["Count"])
    result_all = Entrez.read(search_all)
    ids_all = result_all['IdList']
    pubs_all = Entrez.efetch(db="pubmed", id=ids_all, rettype='medline', retmode='text')
    records = Medline.parse(pubs_all)
    
    acceptable_formats = ["journal article", "comparative study", "editorial"]
    
    for record in records:
        if any([type_.lower() in acceptable_formats for type_ in record.get('PT')]):
            pmid = record.get("PMID")
            pmcid = record.get("PMC", "")
            
            doi = [aid for aid in record.get("AID", []) if aid.endswith(" [doi]")]
            if doi:
                doi = doi[0].replace(" [doi]", "")
            else:
                doi = ""
            
            title = record.get("TI").rstrip(".")
            authors = record.get("AU")
            
            pub_date = parser.parse(record.get("DP"))
            journal = record.get('TA')
            volume = record.get('VI', '')
            issue = record.get('IP', '')
            pages = record.get('PG', '')
            
            row = [pmid, pmcid, doi, title, authors, pub_date.year, pub_date.month,
                   pub_date.day, journal, volume, issue, pages]
            rows += [row]
            
df = pd.DataFrame(columns=['pmid', 'pmcid', 'doi', 'title', 'authors',
                           'year', 'month', 'day',
                           'journal', 'volume', 'issue', 'pages'],
                  data=rows)
df = df[~df["pmid"].isin(skip_pmids)]
df['pmid'] = df['pmid'].astype(int)

new_pubs = df[~df['pmid'].isin(df_old['pmid'])]

# Append to old pubs to solve date issue
df = df_old.append(new_pubs)
df = df.sort_values(by=['year', 'month', 'day'], ascending=False)
df = df.fillna('')

# Save all relevant info from articles to a csv.
print("Saving identified publications to csv...")
df.to_csv('_data/publications/publications.csv', index=False)

Total number of publications containing "Khan AR"[AUTH] AND ("2015/01/01"[PDAT] : "3000/12/31"[PDAT]) AND
                    ("Western University"[AFFL] OR "University of Western Ontario"[AFFL] OR
                     "Brain and Mind Institute"[AFFL] OR "Robarts Research Institute"[AFFL]): 91


Saving identified publications to csv...


/tmp/ipykernel_2363/990535001.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_old.append(new_pubs)


In [5]:
# Add papers we already have pages for.
if len(skip_pmids) > 0:
    for pmid in skip_pmids:
        df = df[df['pmid'] != pmid]
        
print(f"{len(df)} total articles found.")
print(f"{len(df) - old_count} new articles found.")

print("\nNew publications found:")
for _, pub in new_pubs.iterrows():
    print(f"Title: {pub['title']} ({pub['journal']})")
    print(f"Authors: {pub['authors']}")
    print(f"Journal (Date): {pub['journal']} ({pub['month']}/{pub['day']}/{pub['year']})\n")


87 total articles found.
16 new articles found.

New publications found:
Title: Imaging of the superficial white matter in health and disease (Imaging Neurosci (Camb))
Authors: ['Van Dyken PC', 'Khan AR', 'Palaniyappan L']
Journal (Date): Imaging Neurosci (Camb) (9/1/2024)

Title: Hippocampal microscopic fractional anisotropy is reduced in temporal lobe epilepsy (Imaging Neurosci (Camb))
Authors: ['Arezza NJJ', 'Abbas HH', 'Chadwick C', 'Johnsrude IS', 'Burneo JG', 'Suller Marti A', 'Khan AR', 'Baron CA']
Journal (Date): Imaging Neurosci (Camb) (9/1/2024)

Title: Neurite Density and Kurtosis in the Gray Matter of People With Early Schizophrenia (Biol Psychiatry Cogn Neurosci Neuroimaging)
Authors: ['Van Dyken PC', 'Khan AR', 'Palaniyappan L']
Journal (Date): Biol Psychiatry Cogn Neurosci Neuroimaging (6/10/2025)

Title: Early life stress impairs hippocampal subfield myelination (Commun Biol)
Authors: ['Nichols ES', 'Karat BG', 'Grace M', 'Bezanson S', 'Khan AR', 'Duerden EG']
Journal (

In [6]:
# Also output the df in case output limit exceeded
new_pubs

,pmid,pmcid,doi,title,authors,year,month,day,journal,volume,issue,pages
0,40800408,PMC12272272,10.1162/imag_a_00221,Imaging of the superficial white matter in hea...,"[Van Dyken PC, Khan AR, Palaniyappan L]",2024,9,1,Imaging Neurosci (Camb),2,,
1,40800345,PMC12290542,10.1162/imag_a_00356,Hippocampal microscopic fractional anisotropy ...,"[Arezza NJJ, Abbas HH, Chadwick C, Johnsrude I...",2024,9,1,Imaging Neurosci (Camb),2,,
2,40505888,,10.1016/j.bpsc.2025.06.001,Neurite Density and Kurtosis in the Gray Matte...,"[Van Dyken PC, Khan AR, Palaniyappan L]",2025,6,10,Biol Psychiatry Cogn Neurosci Neuroimaging,,,
3,40404790,PMC12098761,10.1038/s42003-025-08165-x,Early life stress impairs hippocampal subfield...,"[Nichols ES, Karat BG, Grace M, Bezanson S, Kh...",2025,5,22,Commun Biol,8,1,785
4,40268921,PMC12019255,10.1038/s41531-025-00916-1,Midbrain and pallidal iron changes identify pa...,"[Alushaj E, Kuurstra A, Menon RS, Ganjavi H, M...",2025,4,23,NPJ Parkinsons Dis,11,1,84
5,40231305,PMC11994302,10.1016/j.bpsgos.2025.100472,Stable White Matter Structure in the First Thr...,"[Van Dyken PC, Yang K, Faria AV, Sawa A, MacKi...",2025,5,1,Biol Psychiatry Glob Open Sci,5,3,100472
6,40202728,PMC11983296,10.1001/jamapsychiatry.2025.0432,Convergence of Cannabis and Psychosis on the D...,"[Ahrens J, Ford SD, Schaefer B, Reese D, Khan ...",2025,6,1,JAMA Psychiatry,82,6,609-617
7,40113845,PMC11926199,10.1038/s41598-025-93033-1,Gauge equivariant convolutional neural network...,"[Hussain U, Khan AR]",2025,3,20,Sci Rep,15,1,9631
8,39420185,PMC11487139,10.1038/s41467-024-53148-x,Multimodal gradients of basal forebrain connec...,"[Chakraborty S, Haast RAM, Onuska KM, Kanel P,...",2024,10,18,Nat Commun,15,1,8990
9,39229062,PMC11370384,10.1101/2024.08.19.608590,The developing hippocampus: Microstructural ev...,"[Karat BG, Genc S, Raven EP, Palombo M, Khan A...",2024,8,19,bioRxiv,,,
